In [1]:
from dipy.reconst import mapmri
from dipy.viz import fvtk
from dipy.data import fetch_cenir_multib, read_cenir_multib, get_sphere
from dipy.core.gradients import gradient_table
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

Download and read the data for this tutorial.

MAPMRI requires multi-shell data, to properly fit the radial part of the basis.
The total size of the downloaded data is 1760 MBytes, however you only need to
fetch it once. Parameter ``with_raw`` of function ``fetch_cenir_multib`` is set
to ``False`` to only download eddy-current/motion corrected data:.

In [4]:
fetch_cenir_multib(with_raw=False)

Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\sesjojen\.dipy\cenir_multib 


({'4D_dwi_eddycor_B200.nii.gz': ('https://digital.lib.washington.edu/researchworks/bitstream/handle/1773/33311/4D_dwi_eddycor_B200.nii.gz',
   'fd704aa3deb83c1c7229202cb3db8c48'),
  '4D_dwieddycor_B1000.nii.gz': ('https://digital.lib.washington.edu/researchworks/bitstream/handle/1773/33311/4D_dwieddycor_B1000.nii.gz',
   'dde8037601a14436b2173f4345b5fd17'),
  '4D_dwieddycor_B2000.nii.gz': ('https://digital.lib.washington.edu/researchworks/bitstream/handle/1773/33311/4D_dwieddycor_B2000.nii.gz',
   'c5e4b96e3afdee99c0e994eff3b2331a'),
  '4D_dwieddycor_B3000.nii.gz': ('https://digital.lib.washington.edu/researchworks/bitstream/handle/1773/33311/4D_dwieddycor_B3000.nii.gz',
   'f0d70456ce424fda2cecd48e64f3a151'),
  '4D_dwieddycor_B400.nii.gz': ('https://digital.lib.washington.edu/researchworks/bitstream/handle/1773/33311/4D_dwieddycor_B400.nii.gz',
   '3d0f2b8ef7b6a4a3aa5c4f7a90c9cfec'),
  'bvals_B1000': ('https://digital.lib.washington.edu/researchworks/bitstream/handle/1773/33311/bvals_

For this example we select only the shell with b-values equal to the one of the
Human Connectome Project (HCP).

For the values of the q-space
indices to make sense it is necessary to explicitly state the big_delta and
small_delta parameters in the gradient table.

In [5]:
bvals = [1000, 2000, 3000]
img, gtab = read_cenir_multib(bvals)
big_delta = 0.0365  # seconds
small_delta = 0.0157  # seconds
gtab = gradient_table(bvals=gtab.bvals, bvecs=gtab.bvecs,
                      small_delta=big_delta,
                      big_delta=small_delta)
data = img.get_data()
data_small = data[40:65, 50:51, 35:60]

print('data.shape (%d, %d, %d, %d)' % data.shape)

Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\sesjojen\.dipy\cenir_multib 
data.shape (104, 104, 72, 612)


Instanstiate the MAPMRI Model.

In [6]:
radial_order = 6
map_model_laplacian_aniso = mapmri.MapmriModel(gtab, radial_order=radial_order,
                                               laplacian_regularization=True,
                                               laplacian_weighting=.2)

We can then fit the MAPMRI model to the data.

In [7]:
mapfit_laplacian_aniso = map_model_laplacian_aniso.fit(data_small)

### Compute the ODFs

Load an odf reconstruction sphere

In [ ]:
sphere = get_sphere('symmetric724')

The radial order s can be increased to sharpen the results, but it might
also make the odfs noisier. Always check the results visually.

In [ ]:
odf = mapfit_both_iso.odf(sphere, s=2)
print('odf.shape (%d, %d, %d, %d)' % odf.shape)

Display the ODFs

In [ ]:
r = fvtk.ren()
sfu = fvtk.sphere_funcs(odf, sphere, colormap='jet')
sfu.RotateX(-90)
fvtk.add(r, sfu)
fvtk.record(r, n_frames=1, out_path='odfs.png', size=(600, 600))